In [5]:
import pandas as pd
import numpy as np
from music21 import converter, midi, interval, pitch
from mido import MidiFile
import miditoolkit
import os
from os import walk
from tokenizing_functions import convert_to_note_items, get_file_and_dirnames
#from helper_functions import get_file_and_dirnames
#from analysis_functions import analyse_data_folder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH_TRANSPOSED = "../0_data/4_preprocessed_sets"

In [96]:
MIN_DURATION_DENOMINATOR = 32
DURATION_STEPS = 64
POSITION_STEPS = 16
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/MIN_DURATION_DENOMINATOR
DURATION_BINS = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*DURATION_STEPS)+1, TICKS_PER_MIN_DURATION, dtype=int)

start_position_tokens = 37 + DURATION_STEPS + 1
end_position_tokens = start_position_tokens + POSITION_STEPS

word2message = {0: "Bar_None"}
for i in range(1, 37):
    word2message[i] = f"Note_on_{i+59}"
for i in range(37, start_position_tokens-1):
    word2message[i] = f"Note_duration_{i-36}"
word2message[start_position_tokens-1] = "Note_duration_triole"
for i in range(start_position_tokens, end_position_tokens):
    word2message[i] = f"Position_{i-start_position_tokens+1}/{POSITION_STEPS}"
word2message[end_position_tokens] = "Position_triole_1"
word2message[end_position_tokens+1] = "Position_triole_2"

message2word = {v: k for k, v in word2message.items()}
message2word

{'Bar_None': 0,
 'Note_on_60': 1,
 'Note_on_61': 2,
 'Note_on_62': 3,
 'Note_on_63': 4,
 'Note_on_64': 5,
 'Note_on_65': 6,
 'Note_on_66': 7,
 'Note_on_67': 8,
 'Note_on_68': 9,
 'Note_on_69': 10,
 'Note_on_70': 11,
 'Note_on_71': 12,
 'Note_on_72': 13,
 'Note_on_73': 14,
 'Note_on_74': 15,
 'Note_on_75': 16,
 'Note_on_76': 17,
 'Note_on_77': 18,
 'Note_on_78': 19,
 'Note_on_79': 20,
 'Note_on_80': 21,
 'Note_on_81': 22,
 'Note_on_82': 23,
 'Note_on_83': 24,
 'Note_on_84': 25,
 'Note_on_85': 26,
 'Note_on_86': 27,
 'Note_on_87': 28,
 'Note_on_88': 29,
 'Note_on_89': 30,
 'Note_on_90': 31,
 'Note_on_91': 32,
 'Note_on_92': 33,
 'Note_on_93': 34,
 'Note_on_94': 35,
 'Note_on_95': 36,
 'Note_duration_1': 37,
 'Note_duration_2': 38,
 'Note_duration_3': 39,
 'Note_duration_4': 40,
 'Note_duration_5': 41,
 'Note_duration_6': 42,
 'Note_duration_7': 43,
 'Note_duration_8': 44,
 'Note_duration_9': 45,
 'Note_duration_10': 46,
 'Note_duration_11': 47,
 'Note_duration_12': 48,
 'Note_duration_13

In [7]:
DURATION_BINS

array([ 128,  256,  384,  512,  640,  768,  896, 1024, 1152, 1280, 1408,
       1536, 1664, 1792, 1920, 2048, 2176, 2304, 2432, 2560, 2688, 2816,
       2944, 3072, 3200, 3328, 3456, 3584, 3712, 3840, 3968, 4096, 4224,
       4352, 4480, 4608, 4736, 4864, 4992, 5120, 5248, 5376, 5504, 5632,
       5760, 5888, 6016, 6144, 6272, 6400, 6528, 6656, 6784, 6912, 7040,
       7168, 7296, 7424, 7552, 7680, 7808, 7936, 8064, 8192])

In [68]:
TRIOLE_DURATIONS = {}
for x in DURATION_BINS:
    x_triole = x/3
    if int(x_triole) != x_triole:
        TRIOLE_DURATIONS[int(x_triole)] = x
        TRIOLE_DURATIONS[int(x_triole)+1] = x
TRIOLE_DURATIONS


{42: 128,
 43: 128,
 85: 256,
 86: 256,
 170: 512,
 171: 512,
 213: 640,
 214: 640,
 298: 896,
 299: 896,
 341: 1024,
 342: 1024,
 426: 1280,
 427: 1280,
 469: 1408,
 470: 1408,
 554: 1664,
 555: 1664,
 597: 1792,
 598: 1792,
 682: 2048,
 683: 2048,
 725: 2176,
 726: 2176,
 810: 2432,
 811: 2432,
 853: 2560,
 854: 2560,
 938: 2816,
 939: 2816,
 981: 2944,
 982: 2944,
 1066: 3200,
 1067: 3200,
 1109: 3328,
 1110: 3328,
 1194: 3584,
 1195: 3584,
 1237: 3712,
 1238: 3712,
 1322: 3968,
 1323: 3968,
 1365: 4096,
 1366: 4096,
 1450: 4352,
 1451: 4352,
 1493: 4480,
 1494: 4480,
 1578: 4736,
 1579: 4736,
 1621: 4864,
 1622: 4864,
 1706: 5120,
 1707: 5120,
 1749: 5248,
 1750: 5248,
 1834: 5504,
 1835: 5504,
 1877: 5632,
 1878: 5632,
 1962: 5888,
 1963: 5888,
 2005: 6016,
 2006: 6016,
 2090: 6272,
 2091: 6272,
 2133: 6400,
 2134: 6400,
 2218: 6656,
 2219: 6656,
 2261: 6784,
 2262: 6784,
 2346: 7040,
 2347: 7040,
 2389: 7168,
 2390: 7168,
 2474: 7424,
 2475: 7424,
 2517: 7552,
 2518: 7552,
 2602:

In [41]:
TRIOLE_DURATION_BINS = np.arange(0, (TICKS_PER_MIN_DURATION*DURATION_STEPS)+1, TICKS_PER_BEAT/4/3, dtype=float)
trioles = []
for x in TRIOLE_DURATION_BINS:
    if int(x) == x:
        trioles.append(int(x))
    else:
        trioles.append(int(x))
        trioles.append(int(x+1))
trioles.remove(0)
for i in DURATION_BINS:
    if i in trioles:
        trioles.remove(i)
trioles

[85,
 86,
 170,
 171,
 341,
 342,
 426,
 427,
 597,
 598,
 682,
 683,
 853,
 854,
 938,
 939,
 1109,
 1110,
 1194,
 1195,
 1365,
 1366,
 1450,
 1451,
 1621,
 1622,
 1706,
 1707,
 1877,
 1878,
 1962,
 1963,
 2133,
 2134,
 2218,
 2219,
 2389,
 2390,
 2474,
 2475,
 2645,
 2646,
 2730,
 2731,
 2901,
 2902,
 2986,
 2987,
 3157,
 3158,
 3242,
 3243,
 3413,
 3414,
 3498,
 3499,
 3669,
 3670,
 3754,
 3755,
 3925,
 3926,
 4010,
 4011,
 4181,
 4182,
 4266,
 4267,
 4437,
 4438,
 4522,
 4523,
 4693,
 4694,
 4778,
 4779,
 4949,
 4950,
 5034,
 5035,
 5205,
 5206,
 5290,
 5291,
 5461,
 5462,
 5546,
 5547,
 5717,
 5718,
 5802,
 5803,
 5973,
 5974,
 6058,
 6059,
 6229,
 6230,
 6314,
 6315,
 6485,
 6486,
 6570,
 6571,
 6741,
 6742,
 6826,
 6827,
 6997,
 6998,
 7082,
 7083,
 7253,
 7254,
 7338,
 7339,
 7509,
 7510,
 7594,
 7595,
 7765,
 7766,
 7850,
 7851,
 8021,
 8022,
 8106,
 8107]

In [98]:
dir = "17_POP909-Dataset-master"
file = "111.mid"
files,_ = get_file_and_dirnames(f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}')
files.sort()
files[:10]

['002.mid',
 '003.mid',
 '004.mid',
 '005.mid',
 '006.mid',
 '007.mid',
 '008.mid',
 '009.mid',
 '010.mid',
 '011.mid']

In [135]:
def split_into_two_bar_items(item, duration, ticks_per_bar=TICKS_PER_BEAT*4):
    items = [] 
    two_bars = duration // (2*ticks_per_bar)
    rest_ticks = duration % (2*ticks_per_bar)
    for i in range(two_bars):
        items.append({
            "name": "Note",
            "start": item.start + i*2*ticks_per_bar,
            "end": item.start + (i+1)*2*ticks_per_bar,
            "pitch": item.pitch,
            "duration": 2*ticks_per_bar
        })
    items.append({
            "name": "Note",
            "start": item.start + two_bars*2*ticks_per_bar,
            "end": item.start + rest_ticks,
            "pitch": item.pitch,
            "duration": rest_ticks
        })
    return items

def convert_to_note_items(path, ticks_per_bar=TICKS_PER_BEAT*4):
    midi_obj = miditoolkit.midi.parser.MidiFile(path)
    notes = midi_obj.instruments[0].notes
    notes.sort(key=lambda x: (x.start, x.pitch))
    note_items = []
    for note in notes:
        duration = note.end - note.start
        if duration > 2*ticks_per_bar:
            split_into_two_bar_items(note, duration, ticks_per_bar)
        else:
            note_items.append({
                "name": "Note",
                "start": note.start,
                "end": note.end,
                "pitch": note.pitch,
                "duration": note.end - note.start
            })
    note_items.sort(key=lambda x: x["start"])
    return note_items

def compute_shifts(items, ticks_per_position=1024*4/POSITION_STEPS):
    #grid
    grids = np.arange(0, items[-1]["start"]+1, ticks_per_position, dtype=int)
    # process
    for item in items:
        index = np.argmin(abs(grids - item["start"]))
        shift = item["start"] - grids[index]
        if shift == 85 or shift == -171:
            item["triole_shift"] = 85
        elif shift == -85:
            item["triole_shift"] = 171
        else:
            item["triole_shift"] = shift
    return items 

def group_items(items, max_time, ticks_per_bar=TICKS_PER_BEAT*4):
    items.sort(key=lambda x: x["start"])
    downbeats = np.arange(0, max_time+ticks_per_bar, ticks_per_bar)
    groups = []
    for db1, db2 in zip(downbeats[:-1], downbeats[1:]):
        insiders = []
        for item in items:
            if (item["start"] >= db1) and (item["start"] < db2):
                insiders.append(item)
        overall = [db1] + insiders + [db2]
        groups.append(overall)
    return groups

def item2event(groups):
    events = []
    n_downbeat = 0
    for i in range(len(groups)):
        if 'Note' not in [item["name"] for item in groups[i][1:-1]]:
            continue
        bar_st, bar_et = groups[i][0], groups[i][-1]
        n_downbeat += 1
        events.append({
            "name": "Bar",
            "time": None,
            "value": None,
            "text": n_downbeat 
        })
        for item in groups[i][1:-1]:

            # position
            flags = np.linspace(bar_st, bar_et, POSITION_STEPS, endpoint=False)
            index = np.argmin(abs(flags-item["start"]))
            if item["start"] < flags[index]:
                index -= 1
            events.append({
                "name": "Position",
                "time": item["start"],
                "value": f"{index+1}/{POSITION_STEPS}",
                "text": flags[index]
            })
            # triole positions
            if item["triole_shift"] == 85:
                events.append({
                    "name": "Position_triole",
                    "time": item["start"],
                    "value": "1",
                    "text": flags[index]+85
                })
            if item["triole_shift"] == 171:
                events.append({
                    "name": "Position_triole",
                    "time": item["start"],
                    "value": "2",
                    "text": flags[index]+171
                })
            # note tokens
            if item["name"] == 'Note':
                # pitch
                events.append({
                    "name": "Note_on",
                    "time": item["start"],
                    "value": item["pitch"],
                    "text": item["pitch"]
                })
                # duration
                duration = item["duration"]
                shift_normal = min(abs(DURATION_BINS-duration))
                shift_triole = min(abs(DURATION_BINS-duration*3))
                if shift_normal <= shift_triole:
                    index = np.argmin(abs(DURATION_BINS-duration))
                    events.append({
                        "name": "Note_duration",
                        "time": item["start"],
                        "value": index+1,
                        "text": f"{duration}/{DURATION_BINS[index]}"
                    })
                else:
                    index = np.argmin(abs(DURATION_BINS-duration*3))
                    events.append({
                        "name": "Note_duration",
                        "time": item["start"],
                        "value": index+1,
                        "text": f"{duration*3}/{DURATION_BINS[index]}"
                    })
                    events.append({
                        "name": "Note_duration",
                        "time": item["start"],
                        "value": "triole",
                        "text": f"{duration}/{DURATION_BINS[index]}"
                    })
    return events


path = f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}/060.mid'
note_items = convert_to_note_items(path)
note_items_shifts = compute_shifts(note_items)

max_time = note_items_shifts[-1]["end"]
grouped_items = group_items(note_items_shifts, max_time)
events = item2event(grouped_items)
events


[{'name': 'Bar', 'time': None, 'value': None, 'text': 1},
 {'name': 'Position', 'time': 24832, 'value': '2/16', 'text': 24832.0},
 {'name': 'Note_on', 'time': 24832, 'value': 79, 'text': 79},
 {'name': 'Note_duration', 'time': 24832, 'value': 4, 'text': '512/512'},
 {'name': 'Position', 'time': 25259, 'value': '3/16', 'text': 25088.0},
 {'name': 'Position_triole', 'time': 25259, 'value': '2', 'text': 25259.0},
 {'name': 'Note_on', 'time': 25259, 'value': 76, 'text': 76},
 {'name': 'Note_duration', 'time': 25259, 'value': 4, 'text': '512/512'},
 {'name': 'Position', 'time': 25856, 'value': '6/16', 'text': 25856.0},
 {'name': 'Note_on', 'time': 25856, 'value': 72, 'text': 72},
 {'name': 'Note_duration', 'time': 25856, 'value': 4, 'text': '512/512'},
 {'name': 'Position', 'time': 26283, 'value': '7/16', 'text': 26112.0},
 {'name': 'Position_triole', 'time': 26283, 'value': '2', 'text': 26283.0},
 {'name': 'Note_on', 'time': 26283, 'value': 69, 'text': 69},
 {'name': 'Note_duration', 'time

In [136]:
def extract_events(input_path):
    note_items = convert_to_note_items(input_path)
    note_items_shifts = compute_shifts(note_items)
    max_time = note_items_shifts[-1]["end"]
    grouped_items = group_items(note_items_shifts, max_time)
    events = item2event(grouped_items)
    return events

events = extract_events(path)
[message2word[f"{e['name']}_{e['value']}"] for e in events][:10]

[0, 103, 20, 40, 104, 119, 17, 40, 107, 13]